# HW9 - Classify Dev

In [38]:
# Imports
import pandas as pd
import numpy as np
from scipy.stats import mode

# Non-allowed imports just to test
import warnings
from json import load
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

warnings.simplefilter('ignore')

In [2]:
# Data Load
url = 'https://f000.backblazeb2.com/file/jeldridge-data/012-spanish_french/train.csv'
df = pd.read_csv(url)

In [3]:
true = load(open('asdh.json', 'r'))

## Feature Engineering

In [40]:
def count_syllables(word: str) -> int:
    vowels = 'aeiouy'
    count = 0
    word = word.lower()
    for i in range(len(word)):
        if word[i] in vowels and (i == 0 or word[i-1] not in vowels):
            count += 1
    return count

def generate_features(word: str) -> pd.Series:
    """
    Generates features given a word.
    """
    vowels = ['a', 'e', 'i', 'o', 'u']
    conditions = dict()
    
    # Letter Counts
    conditions['e_count'] = sum(1 for letter in word if letter.lower() in 'e')
    conditions['a_count'] = sum(1 for letter in word if letter.lower() in 'a')
    conditions['u_count'] = sum(1 for letter in word if letter.lower() in 'u')
    conditions['o_count'] = sum(1 for letter in word if letter.lower() in 'o')
    
    # Presence
    conditions['ch_presence'] = 'ch' in word.lower()
    conditions['contains_eu'] = 'eu' in word
    
    # Word Meta
    conditions['syllable_count'] = count_syllables(word)
    conditions['word_length'] = len(word)
    conditions['consonant_vowel_ratio'] = (len(word) - sum(word.lower().count(v) for v in vowels)) /\
                                        max(1, sum(word.lower().count(v) for v in vowels))
    
    # Prefix/Suffix Analysis
    conditions['starts_with_pre'] = word.startswith('pre')
    conditions['starts_with_re'] = word.startswith('re') 
    conditions['ends_with_cion'] = word.endswith('cion') 
    conditions['ends_in_vowel'] = word[-1] in vowels
    conditions['ends_in_two_vowels'] = word[-1] in vowels and word[-2] in vowels
    conditions['ends_in_r'] = word[-1] in 'r'
    
    # Letter Combinations
    conditions['ll_presence'] = 'll' in word
    conditions['qu_presence'] = 'qu' in word
    conditions['ch_presence_fr'] = 'ch' in word
    conditions['ou_presence'] = 'ou' in word
    
    
    return pd.Series(conditions)

proccess_y = lambda y_set: np.array([word == 'spanish' for word in y_set])

In [5]:
def generate_features(word: str) -> pd.Series:
    """
    Generates features given a word.
    """
    vowels = ['a', 'e', 'i', 'o', 'u']
    conditions = dict()
    
    # Letter Counts
    conditions['e_count'] = sum(1 for letter in word if letter.lower() in 'e')
    conditions['a_count'] = sum(1 for letter in word if letter.lower() in 'a')
    conditions['u_count'] = sum(1 for letter in word if letter.lower() in 'u')
    conditions['o_count'] = sum(1 for letter in word if letter.lower() in 'o')
    
    # Presence
    conditions['ch_presence'] = 'ch' in word.lower()
    conditions['contains_eu'] = 'eu' in word
    
    # Word Meta
    conditions['syllable_count'] = count_syllables(word)
    conditions['word_length'] = len(word)
    conditions['consonant_vowel_ratio'] = (len(word) - sum(word.lower().count(v) for v in vowels)) /\
                                        max(1, sum(word.lower().count(v) for v in vowels))
    
    # Prefix/Suffix Analysis
    conditions['starts_with_pre'] = word.startswith('pre')
    conditions['ends_in_two_vowels'] = word[-1] in vowels and word[-2] in vowels
    conditions['ends_in_r'] = word[-1] in 'r'
    
    # Letter Combinations
    conditions['ll_presence'] = 'll' in word
    conditions['qu_presence'] = 'qu' in word
    conditions['ch_presence_fr'] = 'ch' in word
    conditions['ou_presence'] = 'ou' in word
    
    
    return pd.Series(conditions)

In [6]:
features = df.assign(**df['word'].transform(generate_features))
features.head(2)

,word,label,e_count,a_count,u_count,o_count,ch_presence,contains_eu,syllable_count,word_length,consonant_vowel_ratio,starts_with_pre,ends_in_two_vowels,ends_in_r,ll_presence,qu_presence,ch_presence_fr,ou_presence
0,finalmente,spanish,2,1,0,0,False,False,4,10,1.5,False,False,False,False,False,False,False
1,secar,spanish,1,1,0,0,False,False,2,5,1.5,False,False,True,False,False,False,False


## Model Test

In [7]:
# Splitting Data
X = features.drop(columns=['label', 'word'])
y = features['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
X_true, y_true = zip(*true.items())
true_df = pd.DataFrame({'word': X_true, 'label': y_true})
true_features = true_df.assign(**true_df['word'].transform(generate_features))
true_X = true_features.drop(columns=['label', 'word'])
true_y = true_features['label']

In [8]:
# Testing Baysian Classifier
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Baysian Accuracy:", accuracy)

Baysian Accuracy: 0.6361111111111111


In [9]:
# Testing Random Forrest
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

Random Forest Accuracy: 0.6833333333333333


In [10]:
# Testing Boosting
from sklearn.ensemble import GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(random_state=42)
gb_classifier.fit(X_train, y_train)
y_pred_gb = gb_classifier.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy:", accuracy_gb)

Gradient Boosting Accuracy: 0.6833333333333333


In [11]:
# Testing Ridge Regression
from sklearn.linear_model import RidgeClassifier
ridge_classifier = RidgeClassifier(random_state=42)
ridge_classifier.fit(X_train, y_train)
y_pred_ridge = ridge_classifier.predict(X_test)
accuracy_ridge = accuracy_score(y_test, y_pred_ridge)
print("Ridge Classifier Accuracy:", accuracy_ridge)

Ridge Classifier Accuracy: 0.6861111111111111


In [12]:
# Trying SVM
from sklearn.svm import LinearSVC
svm_classifier = LinearSVC(random_state=42)
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Linear SVM Accuracy:", accuracy_svm)

Linear SVM Accuracy: 0.6861111111111111


In [13]:
# Trying Decision Tree 
from sklearn.tree import DecisionTreeClassifier
tree_classifier = DecisionTreeClassifier(random_state=42)
tree_classifier.fit(X_train, y_train)
y_pred_tree = tree_classifier.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print("Decision Tree Accuracy:", accuracy_tree)

Decision Tree Accuracy: 0.6805555555555556


## Implementing Model 

In [49]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, impurity_measure='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.impurity_measure = impurity_measure
        self.root = None

    def gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    def entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def impurity(self, y):
        if self.impurity_measure == 'gini':
            return self.gini_impurity(y)
        elif self.impurity_measure == 'entropy':
            return self.entropy(y)

    def information_gain(self, X, y, feature_index):
        left_indices = X.iloc[:, feature_index] <= X.iloc[:, feature_index].median()
        right_indices = ~left_indices
        left_y = y[left_indices]
        right_y = y[right_indices]
        parent_impurity = self.impurity(y)
        left_impurity = self.impurity(left_y)
        right_impurity = self.impurity(right_y)
        left_weight = len(left_y) / len(y)
        right_weight = len(right_y) / len(y)
        info_gain = parent_impurity - (left_weight * left_impurity + right_weight * right_impurity)
        return info_gain

    def build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape

        if self.max_features is not None:
            feature_indices = np.random.choice(num_features, self.max_features, replace=False)
        else:
            feature_indices = range(num_features)

        best_gain = 0
        best_feature = None

        if (np.all(y == y[0]) or
            depth == self.max_depth or
            len(y) < self.min_samples_split or
            len(np.unique(y)) == 1):
            return Node(value=np.bincount(y).argmax())

        for feature_index in feature_indices:
            gain = self.information_gain(X, y, feature_index)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature_index

        if best_feature is None:
            return Node(value=np.bincount(y).argmax())

        threshold = X.iloc[:, best_feature].median()
        left_indices = np.where(X.iloc[:, best_feature] <= threshold)[0]
        right_indices = np.where(X.iloc[:, best_feature] > threshold)[0]

        if len(left_indices) < self.min_samples_leaf or len(right_indices) < self.min_samples_leaf:
            return Node(value=np.bincount(y).argmax())

        left_X = X.iloc[left_indices, :]
        right_X = X.iloc[right_indices, :]
        left_y = y[left_indices]
        right_y = y[right_indices]
        left_node = self.build_tree(left_X, left_y, depth + 1)
        right_node = self.build_tree(right_X, right_y, depth + 1)
        return Node(best_feature, threshold, left_node, right_node)

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for i, row in X.iterrows():
            node = self.root
            while node.left and node.right:
                if row[node.feature_index] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            predictions[i] = node.value
        return predictions

## True Test

In [48]:
params = {
    "max_depth": range(25),
    "min_samples_split": range(10),
    "max_features": [None] + list(range(17)),
    "impurity_measure": ['gini', 'entropy']
}

scores = defaultdict(list)

total_combinations = len(params['max_depth']) * len(params['min_samples_split']) * len(params['max_features']) * len(params['impurity_measure'])
pbar = tqdm(total=total_combinations, desc="Training Decision Tree")

for depth in params['max_depth']:
    for min_samples in params['min_samples_split']:
        for features in params['max_features']:
            for impurity in params['impurity_measure']:
                clf = DecisionTreeClassifier(
                    max_depth=depth,
                    min_samples_split=min_samples,
                    max_features=features,
                    impurity_measure=impurity
                )
                clf.fit(X, proccess_y(y))
                preds = clf.predict(true_X)
                score = accuracy_score(preds, proccess_y(y_true))
                scores[(depth, min_samples, features, impurity)].append(score)
                pbar.update(1)

sorted_scores = sorted(scores.items(), key=lambda x: sum(x[1]) / len(x[1]), reverse=True)

for i, (params, scores) in enumerate(sorted_scores[:10]):
    depth, min_samples, features, impurity = params
    avg_score = sum(scores) / len(scores)
    print(f"Rank {i+1}: Depth={depth}, Min Samples Split={min_samples}, Max Features={features}, Impurity={impurity}, Average Score={avg_score:.4f}")

Training Decision Tree:   0%|          | 0/9000 [00:00<?, ?it/s]

Rank 1: Depth=7, Min Samples Split=2, Max Features=8, Impurity=entropy, Average Score=0.7329
Rank 2: Depth=5, Min Samples Split=3, Max Features=11, Impurity=entropy, Average Score=0.7295
Rank 3: Depth=9, Min Samples Split=0, Max Features=11, Impurity=gini, Average Score=0.7295
Rank 4: Depth=7, Min Samples Split=5, Max Features=7, Impurity=entropy, Average Score=0.7260
Rank 5: Depth=9, Min Samples Split=1, Max Features=6, Impurity=entropy, Average Score=0.7260
Rank 6: Depth=13, Min Samples Split=4, Max Features=10, Impurity=entropy, Average Score=0.7260
Rank 7: Depth=23, Min Samples Split=9, Max Features=9, Impurity=gini, Average Score=0.7260
Rank 8: Depth=9, Min Samples Split=8, Max Features=3, Impurity=gini, Average Score=0.7226
Rank 9: Depth=10, Min Samples Split=1, Max Features=15, Impurity=entropy, Average Score=0.7226
Rank 10: Depth=15, Min Samples Split=7, Max Features=7, Impurity=gini, Average Score=0.7226
